In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor


In [2]:
import glob
import pandas as pd
from matplotlib import pyplot as plt
import shutil


In [3]:
def string2params(string, show=True):
    s1 = string.partition("g")
    t = s1[0][1:]
    s2 = s1[2].partition("v")
    g = s2[0]
    v = s2[-1]
    if show:
        print(s1)
        print("t:",t)
        print(s2)
        print("g: ",float(g)/10)
        print("v:", v)
    return float(t), float(g)/10, float(v)

In [4]:
files = glob.glob('filtered_wv_models/*')
m = len(files)
print("Number of files in models", m)


Number of files in models 5425


In [5]:
y = np.zeros((m,3))
df = pd.read_csv(files[0], sep=" ", header=None)
x_points = df[0].values
n_points, n_columns = df.shape
display(df[1].values.shape)
print("Num. of points in x axis: ", n_points)

(201,)

Num. of points in x axis:  201


In [6]:
# X matrix contains the spectral lines
X = np.zeros((m,n_points))
# y matrix contains the labels of each spectral line
y = np.zeros((m,3))
for i,name in enumerate(files):
    file_name = name.split("/")[-1].split(".")[0]
    #print(file_name)
    df = pd.read_csv(name, sep=" ", header=None)
    #print(file_name.split("_")[1])
    t,g,v = string2params(file_name.split("_")[1], show=False)
    X[i,:] = df[1].values
    y[i] = t,g,v
    #print("----")

In [7]:
def train_val_test_split(X, y, train_size, val_size, test_size, random_state = 1):
    if train_size + val_size + test_size != 1.0:
        print("Incorrect sizes!")
        return None
    
    X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size = test_size, random_state=random_state)
    X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size = val_size/(test_size + train_size), random_state=random_state)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y, .8, .1, .1)

In [9]:
rf = RandomForestRegressor(random_state = 42)

from pprint import pprint

# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [10]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor(random_state = 42)
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,
                              n_iter = 8, scoring='neg_mean_absolute_error', 
                              cv = 3, verbose=2, random_state=42, n_jobs=-1,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train);

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [ ]:
rf_random.best_params_

In [ ]:
max_depth = 3
regr_multirf = MultiOutputRegressor(
    RandomForestRegressor(n_estimators=3, max_depth=max_depth, random_state=0)
)
regr_multirf.fit(X_train, y_train)

In [ ]:
y_multirf = regr_multirf.predict(X_test)

In [ ]:
y_multirf

In [ ]:
r_score= regr_multirf.score(X_test,y_test)

In [ ]:
r_score

In [ ]:
def mard(y_test, y_multirf):
    #y_test, y_multirf = np.array(y_test), np.array(y_multirf)
    return (np.mean(np.abs(y_test - y_multirf)/y_multirf))

In [ ]:
mard(y_test[:,0], y_multirf[:,0])

In [ ]:
from sklearn.metrics import mean_absolute_error
print("MAE Teff")
mean_absolute_error(y_test[:,0], y_multirf[:,0])

In [ ]:
print("MAE Log(g)")
mean_absolute_error(y_test[:,1], y_multirf[:,1])

In [ ]:
mard(y_test[:,1], y_multirf[:,1])

In [ ]:
print("MAE Vroot")
mean_absolute_error(y_test[:,2], y_multirf[:,2])

In [ ]:
mard(y_test[:,1], y_multirf[:,1])

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

sns.set_theme(style="whitegrid")

plt.figure()
plt.hist(y_test[:,0], label="Testing")
plt.hist(y_multirf[:,0], label="Pred", histtype='step')
plt.title("T_eff")
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure()
plt.hist(y_test[:,1], label="Testing")
plt.hist(y_multirf[:,1], label="Pred", histtype='step')
plt.title("Log g")
plt.legend(loc="best")
plt.show()

In [ ]:
plt.figure()
plt.hist(y_test[:,2], label="Testing")
plt.hist(y_multirf[:,2], label="Pred", histtype='step')
plt.title("v_rot")
plt.legend(loc="best")
plt.show()

In [ ]:
df = pd.read_csv("BESOS/2_Be_stars/HD33328/PUCHEROS/hd33328_2013-02-26_00-55-34_final_corr.txt", sep="\t", header=None)

m1 = df[0] >= 4460
m2 = df[0] <= 4480

df2 = df[m1][m2]

n = 201  # for 2 random indices
index = np.random.choice(df2[0].shape[0], n, replace=False) 

In [ ]:
index.sort()

In [ ]:
obs_wave = df2[0].values[index]
obs_flux = df2[1].values[index]

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(obs_wave, obs_flux, label="Línea observada (201 pts)")
#plt.plot(df4[0], df4[1], label="Modelo")
plt.legend(loc="best")
plt.show()

In [ ]:
pred_obs = regr_multirf.predict(obs_flux.reshape(1,201))

In [ ]:
pred_obs

In [ ]:
print(pred_obs[0,0])
print(pred_obs[0,1])
print(pred_obs[0,2])

In [ ]:
X.min()

In [ ]:
X.max()